In [1]:
import pandas as pd
import numpy as np 


In [38]:
shd = pd.read_csv("shd_analysis.csv")
shd.head()

,Unnamed: 0,bbl,standard_address,city_id,city_name,boro_id,boro_name,cd_id,cd_name,sba_id,...,end_nep,end_nrp,end_nycha_mix,end_nycha_ph,end_plp,end_prac_202,end_proj8,end_rad,end_tpt,end_year15
0,1,1000160015,50 Battery Place,1,New York City,1,Manhattan,101,MN 01 - Financial District,3810.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1000160020,70 Battery Place,1,New York City,1,Manhattan,101,MN 01 - Financial District,3810.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1000160200,400 Chambers Street,1,New York City,1,Manhattan,101,MN 01 - Financial District,3810.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1000160210,399 Chambers Street,1,New York City,1,Manhattan,101,MN 01 - Financial District,3810.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1000787510,110 Fulton Street,1,New York City,1,Manhattan,101,MN 01 - Financial District,3810.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
shd_program_flag = shd.loc[:,'sub_fin':'sub_zone'].astype('int')

shd_program_flag['subsidy_flag'] = shd_program_flag.loc[:,'sub_fin':'sub_zone'].sum(axis=1)
shd_program_flag = shd_program_flag.replace([0, 1, 2, 3, 4], [0, 1, 1, 1, 1])
shd_program_flag.head()


,sub_fin,sub_prog,sub_tax,sub_zone,subsidy_flag
0,1,0,0,0,1
1,1,0,0,0,1
2,1,0,0,0,1
3,1,0,0,0,1
4,1,0,1,1,1


In [118]:
shd_new = shd.loc[:,['bbl','tract_10']]
shd_new['subsidy_flag'] = 1


In [119]:
shd_start_date = shd.loc[:,'start_202_8':'start_year15']
shd_start_date = shd_start_date.fillna(20200101)
shd_start_date = shd_start_date.astype(int)
shd_start_date['earliest_date'] = shd_start_date.loc[:,'start_202_8':'start_year15'].min(axis=1)


shd_end_date = shd.loc[:,'end_202_8':'end_year15']
shd_end_date = shd_end_date.fillna(0)
shd_end_date = shd_end_date.astype(int)
shd_end_date['latest_date'] = shd_end_date.loc[:,'end_202_8':'end_year15'].max(axis=1)

shd_new['start_earliest'] = shd_start_date['earliest_date'] 
shd_new['end_latest'] = shd_end_date['latest_date']

shd_new = shd_new.replace(0,20301231)
shd_new.head()

,bbl,tract_10,subsidy_flag,start_earliest,end_latest
0,1000160015,36061031704,1,20000101,20301231
1,1000160020,36061031704,1,19990101,20291231
2,1000160200,36061031703,1,19990101,20291231
3,1000160210,36061031703,1,19980101,20281231
4,1000787510,36061001502,1,20070328,20381231


In [120]:
shd_new['start_earliest_time'] = shd_new['start_earliest'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
shd_new['end_latest_time'] = shd_new['end_latest'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
shd_new.head()

,bbl,tract_10,subsidy_flag,start_earliest,end_latest,start_earliest_time,end_latest_time
0,1000160015,36061031704,1,20000101,20301231,2000-01-01,2030-12-31
1,1000160020,36061031704,1,19990101,20291231,1999-01-01,2029-12-31
2,1000160200,36061031703,1,19990101,20291231,1999-01-01,2029-12-31
3,1000160210,36061031703,1,19980101,20281231,1998-01-01,2028-12-31
4,1000787510,36061001502,1,20070328,20381231,2007-03-28,2038-12-31


In [121]:
shd_new.to_csv("shd_clean.csv", index=False)

In [129]:
housing_projects = pd.read_csv("Housing_New_York_Units_by_Building.csv", dtype={'BIN': 'str', 'BBL': 'str'})
housing_projects = housing_projects.dropna(subset=['BBL'])
housing_projects.head()

,Project ID,Project Name,Project Start Date,Project Completion Date,Building ID,Number,Street,Borough,Postcode,BBL,...,2-BR Units,3-BR Units,4-BR Units,5-BR Units,6-BR+ Units,Unknown-BR Units,Counted Rental Units,Counted Homeownership Units,All Counted Units,Total Units
0,66770,146 BAYARD STREET,03/29/2019,NaN,982702.0,146,BAYARD STREET,Brooklyn,11222.0,3027240018,...,4,0,0,0,0,0,8,0,8,46
1,68379,335 FENIMORE STREET APARTMENTS,03/29/2019,NaN,984816.0,335,FENIMORE STREET,Brooklyn,11225.0,3050390055,...,2,0,0,0,0,0,4,0,4,13
2,61271,DOE. 555 NEREID AVE. MULLER,03/28/2019,NaN,99387.0,555,NEREID AVENUE,Bronx,10470.0,2050650001,...,0,0,0,0,0,0,90,0,90,90
3,67051,56 AINSLIE STREET,03/28/2019,NaN,983655.0,56,AINSLIE STREET,Brooklyn,11211.0,3023740016,...,0,0,0,0,0,0,5,0,5,27
6,60784,105TH STREET & AMSTERDAM AVENUE PROJECT,03/27/2019,NaN,6097.0,981,AMSTERDAM AVENUE,Manhattan,10025.0,1018630001,...,13,0,0,0,0,0,0,13,13,13


In [130]:
col_list = ['Project Start Date', 'BBL', 'BIN', 'Borough','Census Tract', "Extremely Low Income Units","Very Low Income Units","Low Income Units","Moderate Income Units","Middle Income Units", "Other Income Units","Counted Rental Units","Counted Homeownership Units","All Counted Units","Total Units"]

In [131]:
housing_projects_simp = housing_projects.loc[:,col_list]
housing_projects_simp.head()

,Project Start Date,BBL,BIN,Borough,Census Tract,Extremely Low Income Units,Very Low Income Units,Low Income Units,Moderate Income Units,Middle Income Units,Other Income Units,Counted Rental Units,Counted Homeownership Units,All Counted Units,Total Units
0,03/29/2019,3027240018,NaN,Brooklyn,499.0,0,0,8,0,0,0,8,0,8,46
1,03/29/2019,3050390055,NaN,Brooklyn,802.0,0,0,0,0,4,0,4,0,4,13
2,03/28/2019,2050650001,2070559,Bronx,414.0,54,0,35,0,0,1,90,0,90,90
3,03/28/2019,3023740016,NaN,Brooklyn,513.0,0,0,5,0,0,0,5,0,5,27
6,03/27/2019,1018630001,1055996,Manhattan,193.0,0,0,0,13,0,0,0,13,13,13


In [132]:
to_cal_list = ["Extremely Low Income Units","Very Low Income Units","Low Income Units","Moderate Income Units","Middle Income Units","Other Income Units"]
housing_projects_simp_result = housing_projects_simp[to_cal_list].div(housing_projects_simp['Total Units'],axis=0)
housing_projects_simp_result['Commercial Units'] = 1 - housing_projects_simp_result.loc[:,"Extremely Low Income Units":"Other Income Units"].sum(axis=1)
housing_projects_simp_result.head()


,Extremely Low Income Units,Very Low Income Units,Low Income Units,Moderate Income Units,Middle Income Units,Other Income Units,Commercial Units
0,0.0,0.0,0.173913,0.0,0.000000,0.000000,0.826087
1,0.0,0.0,0.000000,0.0,0.307692,0.000000,0.692308
2,0.6,0.0,0.388889,0.0,0.000000,0.011111,0.000000
3,0.0,0.0,0.185185,0.0,0.000000,0.000000,0.814815
6,0.0,0.0,0.000000,1.0,0.000000,0.000000,0.000000


In [133]:
housing_projects_simp_info = housing_projects_simp.loc[:,['Project Start Date', 'BBL', 'BIN', 'Borough']]
housing_projects_final = pd.concat([housing_projects_simp_info, housing_projects_simp_result], axis=1)


In [135]:
housing_projects_summary = housing_projects_final
housing_projects_summary['< Low Income Units'] = housing_projects_summary.loc[:,'Extremely Low Income Units':'Low Income Units'].sum(axis=1)
housing_projects_summary = housing_projects_summary.drop(columns=['Borough','Extremely Low Income Units', 'Very Low Income Units','Low Income Units'])

In [136]:
housing_projects_summary.head()

,Project Start Date,BBL,BIN,Moderate Income Units,Middle Income Units,Other Income Units,Commercial Units,< Low Income Units
0,03/29/2019,3027240018,NaN,0.0,0.000000,0.000000,0.826087,0.173913
1,03/29/2019,3050390055,NaN,0.0,0.307692,0.000000,0.692308,0.000000
2,03/28/2019,2050650001,2070559,0.0,0.000000,0.011111,0.000000,0.988889
3,03/28/2019,3023740016,NaN,0.0,0.000000,0.000000,0.814815,0.185185
6,03/27/2019,1018630001,1055996,1.0,0.000000,0.000000,0.000000,0.000000


In [173]:
housing_projects_summary['affordability'] = 0
n = housing_projects_summary.index.tolist()
for i in range(0,len(housing_projects_summary)):
    max_index = 3
    list_new = housing_projects_summary.iloc[i,3:8].tolist()
    temp = list_new[0]
    max_index = 0
    for j in range(0,len(list_new)):
        temp_1 = list_new[j]
        if(temp_1 > temp):
            temp = temp_1
            max_index = j
    #print temp
    housing_projects_summary.iloc[i,8] = max_index    
        
housing_projects_summary.head()      
   
            
        
        

,Project Start Date,BBL,BIN,Moderate Income Units,Middle Income Units,Other Income Units,Commercial Units,< Low Income Units,affordability
0,03/29/2019,3027240018,NaN,0.0,0.000000,0.000000,0.826087,0.173913,3
1,03/29/2019,3050390055,NaN,0.0,0.307692,0.000000,0.692308,0.000000,3
2,03/28/2019,2050650001,2070559,0.0,0.000000,0.011111,0.000000,0.988889,4
3,03/28/2019,3023740016,NaN,0.0,0.000000,0.000000,0.814815,0.185185,3
6,03/27/2019,1018630001,1055996,1.0,0.000000,0.000000,0.000000,0.000000,0


In [175]:
housing_projects_summary.affordability = housing_projects_summary.affordability.replace([0, 1, 2, 3, 4], [3, 2, 1, 0, 4])
housing_projects_summary.head()  

,Project Start Date,BBL,BIN,Moderate Income Units,Middle Income Units,Other Income Units,Commercial Units,< Low Income Units,affordability
0,03/29/2019,3027240018,NaN,0.0,0.000000,0.000000,0.826087,0.173913,0
1,03/29/2019,3050390055,NaN,0.0,0.307692,0.000000,0.692308,0.000000,0
2,03/28/2019,2050650001,2070559,0.0,0.000000,0.011111,0.000000,0.988889,4
3,03/28/2019,3023740016,NaN,0.0,0.000000,0.000000,0.814815,0.185185,0
6,03/27/2019,1018630001,1055996,1.0,0.000000,0.000000,0.000000,0.000000,3


In [176]:
housing_projects_summary['year'] = housing_projects_summary['Project Start Date'].apply(lambda x: x[-4:])
housing_projects_summary.head()

,Project Start Date,BBL,BIN,Moderate Income Units,Middle Income Units,Other Income Units,Commercial Units,< Low Income Units,affordability,year
0,03/29/2019,3027240018,NaN,0.0,0.000000,0.000000,0.826087,0.173913,0,2019
1,03/29/2019,3050390055,NaN,0.0,0.307692,0.000000,0.692308,0.000000,0,2019
2,03/28/2019,2050650001,2070559,0.0,0.000000,0.011111,0.000000,0.988889,4,2019
3,03/28/2019,3023740016,NaN,0.0,0.000000,0.000000,0.814815,0.185185,0,2019
6,03/27/2019,1018630001,1055996,1.0,0.000000,0.000000,0.000000,0.000000,3,2019


In [179]:
housing_projects_output = housing_projects_summary.loc[:,[ 'year','BBL', 'BIN', 'affordability']]
housing_projects_output.to_csv("housing_projects_output.csv", index = False)